# Africa Risk Score Analysis

This notebook fetches risk scores for **Drought, Flood, and Wildfire** for African countries from the GO Risk API.

**Data Sources:**
- Region 0: Sub-Saharan Africa
- Region 4: North Africa (filtered to Morocco, Tunisia, Algeria, Libya, Egypt)

**Classification Systems:**
| Score Range | GO Platform # | Category |
|------------|---------------|----------|
| 0.0 - 2.0 | 1 | Very Low |
| 2.0 - 3.5 | 2 | Low |
| 3.5 - 5.0 | 3 | Medium |
| 5.0 - 6.5 | 4 | High |
| 6.5 - 10.0 | 5 | Very High |

In [2]:
import requests
import pandas as pd
from datetime import datetime

# Configuration
NORTH_AFRICA_COUNTRIES = ['Morocco', 'Tunisia', 'Algeria', 'Libya', 'Egypt']
INCLUDED_HAZARDS = ['Flood', 'Drought']

## 1. Data Fetching Functions

In [122]:
def fetch_risk_scores(region=0, limit=9999):
    """Fetch risk scores from GO Risk API for a specific region."""
    url = f"https://go-risk.northeurope.cloudapp.azure.com/api/v1/risk-score/?region={region}&limit={limit}"
    print(f"Fetching data from: {url}")
    
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"API request failed with status code: {response.status_code}")
    
    data = response.json()
    print(f"Successfully fetched {data['count']} records")
    return data['results']


def fetch_all_africa_data():
    """Fetch risk scores for all of Africa (Sub-Saharan + North Africa)."""
    # Fetch Sub-Saharan Africa (region 0)
    print("--- Fetching Sub-Saharan Africa (region=0) ---")
    sub_saharan_results = fetch_risk_scores(region=0)
    print(f"Sub-Saharan Africa records: {len(sub_saharan_results)}")
    
    # Fetch North Africa (region 4)
    print("\n--- Fetching North Africa (region=4) ---")
    north_africa_results = fetch_risk_scores(region=4)
    print(f"North Africa records (before filtering): {len(north_africa_results)}")
    
    # Filter to specific North African countries
    filtered_north_africa = [
        record for record in north_africa_results
        if record['country_details']['name'] in NORTH_AFRICA_COUNTRIES
    ]
    print(f"North Africa records (after filtering): {len(filtered_north_africa)}")
    
    # Combine datasets
    all_africa_results = sub_saharan_results + filtered_north_africa
    print(f"\nTotal Africa records: {len(all_africa_results)}")
    
    return all_africa_results


def filter_by_hazard_type(results, hazard_types=None):
    """Filter results to only include specified hazard types."""
    if hazard_types is None:
        hazard_types = INCLUDED_HAZARDS
    
    filtered = [r for r in results if r['hazard_type_display'] in hazard_types]
    print(f"Filtered to hazards {hazard_types}: {len(filtered)} records")
    return filtered

## 2. Classification Functions

In [123]:
def categorize_go_platform(score):
    """Categorize using GO Platform methodology (1-5 scale)."""
    if score is None or pd.isna(score):
        return (0, "No Data")
    elif score < 2.0:
        return (1, "Very Low")
    elif score < 3.5:
        return (2, "Low")
    elif score < 5.0:
        return (3, "Medium")
    elif score < 6.5:
        return (4, "High")
    else:
        return (5, "Very High")


def categorize_inform(score):
    """Categorize using INFORM methodology (text-based)."""
    if score is None or pd.isna(score):
        return "No Data"
    elif score < 2.0:
        return "Very Low"
    elif score < 3.5:
        return "Low"
    elif score < 5.0:
        return "Medium"
    elif score < 6.5:
        return "High"
    else:
        return "Very High"

## 3. Data Processing Functions

In [124]:
def process_risk_data(results):
    """Process raw API results into structured data with calculations."""
    processed_data = []
    months = ['january', 'february', 'march', 'april', 'may', 'june',
              'july', 'august', 'september', 'october', 'november', 'december']
    
    for record in results:
        country_name = record['country_details']['name']
        country_iso3 = record['country_details']['iso3'].upper()
        hazard_type = record['hazard_type_display']
        
        # Get monthly scores
        monthly_scores = [record.get(month, 0) or 0 for month in months]
        
        # Calculate averages
        yearly_sum = record.get('yearly_sum', sum(monthly_scores))
        yearly_avg = yearly_sum / 12 if yearly_sum else 0
        peak_month_score = max(monthly_scores) if monthly_scores else 0
        
        # Apply classifications
        go_cat_num, go_cat_name = categorize_go_platform(yearly_avg)
        inform_category = categorize_inform(yearly_avg)
        peak_go_num, peak_go_name = categorize_go_platform(peak_month_score)
        peak_inform = categorize_inform(peak_month_score)
        
        processed_data.append({
            'Country': country_name,
            'ISO3': country_iso3,
            'Hazard_Type': hazard_type,
            'January': monthly_scores[0], 'February': monthly_scores[1],
            'March': monthly_scores[2], 'April': monthly_scores[3],
            'May': monthly_scores[4], 'June': monthly_scores[5],
            'July': monthly_scores[6], 'August': monthly_scores[7],
            'September': monthly_scores[8], 'October': monthly_scores[9],
            'November': monthly_scores[10], 'December': monthly_scores[11],
            'Yearly_Sum': round(yearly_sum, 2),
            'Yearly_Average': round(yearly_avg, 2),
            'Peak_Month_Score': round(peak_month_score, 2),
            'GO_Category_Num': go_cat_num,
            'GO_Category': go_cat_name,
            'INFORM_Category': inform_category,
            'Peak_GO_Num': peak_go_num,
            'Peak_GO_Category': peak_go_name,
            'Peak_INFORM_Category': peak_inform,
            'LCC': record.get('lcc', 0),
            'Vulnerability': record.get('vulnerability', 0),
            'Population_Thousands': record.get('population_in_thousands', 0)
        })
    
    return processed_data


def create_summary_by_country(df):
    """Create a summary view with all hazard types per country in columns."""
    summary_data = []
    
    for country in sorted(df['Country'].unique()):
        country_data = df[df['Country'] == country]
        
        row = {
            'Country': country,
            'ISO3': country_data['ISO3'].iloc[0],
            'Population_Thousands': country_data['Population_Thousands'].iloc[0],
            'LCC': country_data['LCC'].iloc[0],
            'Vulnerability': country_data['Vulnerability'].iloc[0],
        }
        
        for hazard in INCLUDED_HAZARDS:
            hazard_row = country_data[country_data['Hazard_Type'] == hazard]
            if not hazard_row.empty:
                row[f'{hazard}_Sum'] = hazard_row['Yearly_Sum'].values[0]
                row[f'{hazard}_Avg'] = hazard_row['Yearly_Average'].values[0]
                row[f'{hazard}_Peak'] = hazard_row['Peak_Month_Score'].values[0]
                row[f'{hazard}_GO_Num'] = hazard_row['GO_Category_Num'].values[0]
                row[f'{hazard}_GO_Cat'] = hazard_row['GO_Category'].values[0]
                row[f'{hazard}_INFORM'] = hazard_row['INFORM_Category'].values[0]
            else:
                row[f'{hazard}_Sum'] = 0
                row[f'{hazard}_Avg'] = 0
                row[f'{hazard}_Peak'] = 0
                row[f'{hazard}_GO_Num'] = 0
                row[f'{hazard}_GO_Cat'] = 'No Data'
                row[f'{hazard}_INFORM'] = 'No Data'
        
        # Calculate overall average
        hazard_avgs = [row.get(f'{h}_Avg', 0) for h in INCLUDED_HAZARDS]
        total_avg = sum(hazard_avgs) / len(INCLUDED_HAZARDS)
        row['Overall_Avg'] = round(total_avg, 2)
        
        go_num, go_cat = categorize_go_platform(total_avg)
        row['Overall_GO_Num'] = go_num
        row['Overall_GO_Cat'] = go_cat
        row['Overall_INFORM'] = categorize_inform(total_avg)
        
        summary_data.append(row)
    
    return pd.DataFrame(summary_data)

## 4. Fetch and Process Data

In [125]:
# Fetch all Africa data
print("=" * 60)
print("FETCHING RISK SCORES")
print("=" * 60)

all_results = fetch_all_africa_data()

# Filter to selected hazard types
print("\n" + "=" * 60)
print("FILTERING HAZARD TYPES")
print("=" * 60)

filtered_results = filter_by_hazard_type(all_results)

FETCHING RISK SCORES
--- Fetching Sub-Saharan Africa (region=0) ---
Fetching data from: https://go-risk.northeurope.cloudapp.azure.com/api/v1/risk-score/?region=0&limit=9999
Successfully fetched 150 records
Sub-Saharan Africa records: 150

--- Fetching North Africa (region=4) ---
Fetching data from: https://go-risk.northeurope.cloudapp.azure.com/api/v1/risk-score/?region=4&limit=9999
Successfully fetched 51 records
North Africa records (before filtering): 51
North Africa records (after filtering): 15

Total Africa records: 165

FILTERING HAZARD TYPES
Filtered to hazards ['Flood', 'Drought']: 110 records


In [126]:
# Process the data
print("Processing data...")
processed_data = process_risk_data(filtered_results)
detailed_df = pd.DataFrame(processed_data)

print(f"\nProcessed {len(detailed_df)} records")
print(f"Countries: {detailed_df['Country'].nunique()}")
print(f"Hazard types: {detailed_df['Hazard_Type'].unique().tolist()}")

Processing data...

Processed 110 records
Countries: 55
Hazard types: ['Drought', 'Flood']


In [127]:
# Create country summary
summary_df = create_summary_by_country(detailed_df)
print(f"Summary created for {len(summary_df)} countries")

Summary created for 55 countries


## 5. View Results

In [128]:
# View first few rows of detailed data
detailed_df.head(10)

,Country,ISO3,Hazard_Type,January,February,March,April,May,June,July,...,Peak_Month_Score,GO_Category_Num,GO_Category,INFORM_Category,Peak_GO_Num,Peak_GO_Category,Peak_INFORM_Category,LCC,Vulnerability,Population_Thousands
0,Angola,AGO,Drought,4.1,3.9,4.1,4.1,3.9,3.9,3.8,...,4.2,3,Medium,Medium,3,Medium,Medium,7.0,5.5,32898.997
1,Angola,AGO,Flood,3.2,2.6,5.1,1.9,0.6,0.3,0.0,...,5.1,1,Very Low,Very Low,4,High,High,7.0,5.5,32898.997
2,Benin,BEN,Drought,0.0,0.0,0.4,0.5,0.4,0.2,0.2,...,0.5,1,Very Low,Very Low,1,Very Low,Very Low,7.0,5.2,12466.819
3,Benin,BEN,Flood,0.0,0.0,0.0,0.0,0.0,2.0,5.1,...,5.1,1,Very Low,Very Low,4,High,High,7.0,5.2,12466.819
4,Botswana,BWA,Drought,5.7,6.2,6.2,5.6,5.7,3.4,1.5,...,6.5,3,Medium,Medium,5,Very High,Very High,5.2,4.0,2523.541
5,Botswana,BWA,Flood,1.2,4.8,3.6,0.0,1.2,1.2,1.2,...,4.8,1,Very Low,Very Low,3,Medium,Medium,5.2,4.0,2523.541
6,Burkina Faso,BFA,Drought,3.8,3.8,3.9,4.2,4.9,5.4,5.8,...,5.8,3,Medium,Medium,4,High,High,7.1,7.4,21231.893
7,Burkina Faso,BFA,Flood,0.0,0.0,0.0,0.0,0.0,2.3,4.6,...,4.6,1,Very Low,Very Low,3,Medium,Medium,7.1,7.4,21231.893
8,Burundi,BDI,Drought,3.4,3.4,3.4,3.4,3.4,3.4,3.4,...,4.9,3,Medium,Medium,3,Medium,Medium,6.3,6.4,12053.897
9,Burundi,BDI,Flood,3.7,3.0,3.0,3.7,3.7,0.7,0.0,...,3.7,1,Very Low,Very Low,3,Medium,Medium,6.3,6.4,12053.897


In [129]:
# View country summary
summary_df.head(10)

,Country,ISO3,Population_Thousands,LCC,Vulnerability,Flood_Sum,Flood_Avg,Flood_Peak,Flood_GO_Num,Flood_GO_Cat,...,Drought_Sum,Drought_Avg,Drought_Peak,Drought_GO_Num,Drought_GO_Cat,Drought_INFORM,Overall_Avg,Overall_GO_Num,Overall_GO_Cat,Overall_INFORM
0,Africa Region,KEN,51460.211,5.7,6.1,25.3,2.11,5.6,2,Low,...,71.0,5.92,7.0,4,High,High,4.01,3,Medium,Medium
1,Algeria,DZA,43090.987,4.4,3.1,24.5,2.04,5.2,2,Low,...,29.9,2.49,4.1,2,Low,Low,2.26,2,Low,Low
2,Angola,AGO,32898.997,7.0,5.5,15.6,1.30,5.1,1,Very Low,...,47.5,3.96,4.2,3,Medium,Medium,2.63,2,Low,Low
3,Benin,BEN,12466.819,7.0,5.2,13.2,1.10,5.1,1,Very Low,...,2.6,0.22,0.5,1,Very Low,Very Low,0.66,1,Very Low,Very Low
4,Botswana,BWA,2523.541,5.2,4.0,13.2,1.10,4.8,1,Very Low,...,50.4,4.20,6.5,3,Medium,Medium,2.65,2,Low,Low
5,Burkina Faso,BFA,21231.893,7.1,7.4,16.8,1.40,4.6,1,Very Low,...,54.9,4.57,5.8,3,Medium,Medium,2.99,2,Low,Low
6,Burundi,BDI,12053.897,6.3,6.4,22.9,1.91,3.7,1,Very Low,...,43.7,3.64,4.9,3,Medium,Medium,2.78,2,Low,Low
7,Cabo Verde,CPV,579.933,4.1,3.3,1.2,0.10,0.1,1,Very Low,...,19.2,1.60,1.6,1,Very Low,Very Low,0.85,1,Very Low,Very Low
8,Cameroon,CMR,26137.129,6.8,6.6,11.1,0.92,6.0,1,Very Low,...,31.3,2.61,3.1,2,Low,Low,1.76,1,Very Low,Very Low
9,Central African Republic,CAF,5272.026,8.8,8.7,20.0,1.67,5.7,1,Very Low,...,1.4,0.12,0.5,1,Very Low,Very Low,0.90,1,Very Low,Very Low


In [130]:
# Top 10 countries by overall risk
print("TOP 10 COUNTRIES BY OVERALL RISK")
print("-" * 50)
top_10 = summary_df.nlargest(10, 'Overall_Avg')[['Country', 'Overall_Avg', 'Overall_GO_Cat']]
top_10

TOP 10 COUNTRIES BY OVERALL RISK
--------------------------------------------------


,Country,Overall_Avg,Overall_GO_Cat
45,Somalia,5.53,High
46,South Africa,4.36,Medium
0,Africa Region,4.01,Medium
26,Kenya,4.01,Medium
38,Niger,4.00,Medium
54,Zimbabwe,3.72,Medium
37,Namibia,3.62,Medium
33,Mauritania,3.47,Low
36,Mozambique,3.44,Low
48,Sudan,3.44,Low


In [131]:
# Risk category distribution per hazard
print("RISK CATEGORY DISTRIBUTION")
print("=" * 50)

for hazard in INCLUDED_HAZARDS:
    print(f"\n{hazard.upper()}:")
    hazard_df = detailed_df[detailed_df['Hazard_Type'] == hazard]
    counts = hazard_df['GO_Category'].value_counts()
    for cat in ['Very Low', 'Low', 'Medium', 'High', 'Very High']:
        print(f"  {cat}: {counts.get(cat, 0)} countries")

RISK CATEGORY DISTRIBUTION

FLOOD:
  Very Low: 47 countries
  Low: 8 countries
  Medium: 0 countries
  High: 0 countries
  Very High: 0 countries

DROUGHT:
  Very Low: 21 countries
  Low: 6 countries
  Medium: 16 countries
  High: 10 countries
  Very High: 2 countries


## 6. Export to Excel

In [132]:
def export_to_excel(detailed_df, summary_df, output_path):
    """Export data to Excel with multiple sheets."""
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        summary_df.to_excel(writer, sheet_name='Country_Summary', index=False)
        detailed_df.to_excel(writer, sheet_name='Detailed_Monthly', index=False)
        
        for hazard in INCLUDED_HAZARDS:
            hazard_df = detailed_df[detailed_df['Hazard_Type'] == hazard].copy()
            hazard_df.to_excel(writer, sheet_name=hazard, index=False)
        
        # Classification reference
        classification_df = pd.DataFrame({
            'Score_Range': ['0.0 - 2.0', '2.0 - 3.5', '3.5 - 5.0', '5.0 - 6.5', '6.5 - 10.0'],
            'GO_Platform_Number': [1, 2, 3, 4, 5],
            'Category': ['Very Low', 'Low', 'Medium', 'High', 'Very High']
        })
        classification_df.to_excel(writer, sheet_name='Classification_Reference', index=False)
    
    print(f"Excel file saved to: {output_path}")


# Export
output_path = "africa_risk_scores_analysis.xlsx"
export_to_excel(detailed_df, summary_df, output_path)

Excel file saved to: africa_risk_scores_analysis.xlsx


## 7. Verify North African Countries

In [133]:
# Check that North African countries are included
print("North African countries in dataset:")
for country in NORTH_AFRICA_COUNTRIES:
    in_data = country in detailed_df['Country'].values
    status = "✓" if in_data else "✗"
    print(f"  {status} {country}")

North African countries in dataset:
  ✓ Morocco
  ✓ Tunisia
  ✓ Algeria
  ✓ Libya
  ✓ Egypt


In [134]:
import plotly.io as pio
pio.renderers.default = "notebook"


In [135]:
import pandas as pd
import numpy as np
import json
import requests
import plotly.express as px
import plotly.graph_objects as go



In [136]:
# ============================================================
# MAPBOX CONFIGURATION - IFRC Custom Tileset
# ============================================================
MAPBOX_TOKEN = "pk.eyJ1IjoiZ28taWZyYyIsImEiOiJjbWYwdXhyaXIxYTk3MnJzNzliM3B4cHozIn0.lu7cqSPDHP4Mm2i00PcBQw"
MAPBOX_STYLE_URL = "mapbox://styles/go-ifrc/ckrfe16ru4c8718phmckdfjh0"



In [137]:
# Vector tile info (for reference)
# VECTOR_TILE = "mapbox://go-ifrc.go-countries"
# SOURCE_LAYER = "go-countries"
# VECTOR_PROPERTY = "iso3"

# Africa GeoJSON URL (with ISO3 codes)
GEOJSON_URL = "https://raw.githubusercontent.com/datasets/geo-countries/master/data/countries.geojson"



In [138]:
# ============================================================
# DATA LOADING
# ============================================================
def load_geojson():
    """Load Africa countries GeoJSON from web."""
    print("Downloading world countries GeoJSON...")
    response = requests.get(GEOJSON_URL)
    world_geojson = response.json()
    
    # Filter to Africa countries (we'll use our data ISO3 codes to match)
    print(f"Loaded GeoJSON with {len(world_geojson['features'])} countries")
    return world_geojson




In [139]:
def load_and_normalize_data(excel_path="africa_risk_scores_analysis.xlsx"):
    """Load the summary data and calculate normalized scores."""
    
    # Read the country summary sheet
    summary_df = pd.read_excel(excel_path, sheet_name='Country_Summary')
    
    viz_df = summary_df.copy()
    
    # Calculate sqrt of population for normalization
    # Formula: Normalized Score = Yearly Average / sqrt(Population in Thousands)
    viz_df['Pop_Sqrt'] = np.sqrt(viz_df['Population_Thousands'])
    
    # Normalize each hazard and overall average
    viz_df['Drought_Norm'] = (viz_df['Drought_Avg'] / viz_df['Pop_Sqrt']).round(4)
    viz_df['Flood_Norm'] = (viz_df['Flood_Avg'] / viz_df['Pop_Sqrt']).round(4)
    viz_df['Overall_Norm'] = (viz_df['Overall_Avg'] / viz_df['Pop_Sqrt']).round(4)
    
    # Remove rows with missing ISO3 codes
    viz_df = viz_df[viz_df['ISO3'].notna() & (viz_df['ISO3'] != '')]
    
    print(f"Loaded {len(viz_df)} countries with normalized scores")
    return viz_df


# ============================================================
# COLOR SCALES
# ============================================================
# Sequential orange color scale (user preference)

DROUGHT_COLOR_SCALE = "Reds"

FLOOD_COLOR_SCALE = "Blues"  # Plotly built-in Blues




In [140]:
# ============================================================
# CHOROPLETH MAP FUNCTIONS (using choropleth_mapbox)
# ============================================================
def create_drought_map(viz_df, geojson_data):
    """Create Drought Risk choropleth map using Mapbox."""
    
    fig = px.choropleth_mapbox(
        viz_df,
        geojson=geojson_data,
        locations='ISO3',
        featureidkey='properties.ISO3166-1-Alpha-3',  # Match to GeoJSON property
        color='Drought_Norm',
        hover_name='Country',
        hover_data={
            'ISO3': True,
            'Drought_Avg': ':.2f',
            'Drought_Norm': ':.4f',
            'Population_Thousands': ':.0f',
            'Drought_GO_Cat': True
        },
        color_continuous_scale=DROUGHT_COLOR_SCALE,
        mapbox_style=MAPBOX_STYLE_URL,
        zoom=2.3,
        center={"lat": 0, "lon": 20},
        opacity=0.7,
    )
    
    fig.update_layout(
        mapbox=dict(accesstoken=MAPBOX_TOKEN),
        height=700,
        width=800,
        title_text="<b>INFORM Drought Risk Score (Population-Normalized)</b>",
        coloraxis_colorbar=dict(
            title="Score",
            orientation="h",
            x=0.5,
            y=-0.1,
            xanchor="center",
            tickvals=[viz_df['Drought_Norm'].min(), viz_df['Drought_Norm'].max()],
            ticktext=["Low", "High"]
        )
    )
    
    return fig


def create_flood_map(viz_df, geojson_data):
    """Create Flood Risk choropleth map using Mapbox."""
    
    fig = px.choropleth_mapbox(
        viz_df,
        geojson=geojson_data,
        locations='ISO3',
        featureidkey='properties.ISO3166-1-Alpha-3',
        color='Flood_Norm',
        hover_name='Country',
        hover_data={
            'ISO3': True,
            'Flood_Avg': ':.2f',
            'Flood_Norm': ':.4f',
            'Population_Thousands': ':.0f',
            'Flood_GO_Cat': True
        },
        color_continuous_scale=FLOOD_COLOR_SCALE,
        mapbox_style=MAPBOX_STYLE_URL,
        zoom=2.3,
        center={"lat": 0, "lon": 20},
        opacity=0.7,
    )
    
    fig.update_layout(
        mapbox=dict(accesstoken=MAPBOX_TOKEN),
        height=700,
        width=800,
        title_text="<b>INFORM Flood Risk Score (Population-Normalized)</b>",
        coloraxis_colorbar=dict(
            title="Score",
            orientation="h",
            x=0.5,
            y=-0.1,
            xanchor="center",
            tickvals=[viz_df['Flood_Norm'].min(), viz_df['Flood_Norm'].max()],
            ticktext=["Low", "High"]
        )
    )
    
    return fig




In [141]:
def create_overall_map(viz_df, geojson_data):
    """Create Overall Risk choropleth map using Mapbox."""
    
    fig = px.choropleth_mapbox(
        viz_df,
        geojson=geojson_data,
        locations='ISO3',
        featureidkey='properties.ISO3166-1-Alpha-3',
        color='Overall_Norm',
        hover_name='Country',
        hover_data={
            'ISO3': True,
            'Overall_Avg': ':.2f',
            'Overall_Norm': ':.4f',
            'Drought_Avg': ':.2f',
            'Flood_Avg': ':.2f',
            'Population_Thousands': ':.0f',
            'Overall_GO_Cat': True
        },
        color_continuous_scale="Reds",
        mapbox_style=MAPBOX_STYLE_URL,
        zoom=2.3,
        center={"lat": 0, "lon": 20},
        opacity=0.7,
    )
    
    fig.update_layout(
        mapbox=dict(accesstoken=MAPBOX_TOKEN),
        height=700,
        width=800,
        title_text="<b>INFORM Risk Score (Population-Normalized)</b>",
        coloraxis_colorbar=dict(
            title="Score",
            orientation="h",
            x=0.5,
            y=-0.1,
            xanchor="center",
            tickvals=[viz_df['Overall_Norm'].min(), viz_df['Overall_Norm'].max()],
            ticktext=["Low", "High"]
        )
    )
    
    return fig


def create_raw_overall_map(viz_df, geojson_data):
    """Create Raw (non-normalized) Overall Risk choropleth for comparison."""
    
    fig = px.choropleth_mapbox(
        viz_df,
        geojson=geojson_data,
        locations='ISO3',
        featureidkey='properties.ISO3166-1-Alpha-3',
        color='Overall_Avg',
        hover_name='Country',
        hover_data={
            'ISO3': True,
            'Overall_Avg': ':.2f',
            'Drought_Avg': ':.2f',
            'Flood_Avg': ':.2f',
            'Population_Thousands': ':.0f'
        },
        color_continuous_scale="Reds",
        mapbox_style=MAPBOX_STYLE_URL,
        zoom=2.3,
        center={"lat": 0, "lon": 20},
        opacity=0.7,
    )
    
    fig.update_layout(
        mapbox=dict(accesstoken=MAPBOX_TOKEN),
        height=700,
        width=800,
        title_text="<b>INFORM Risk Score</b>",
        coloraxis_colorbar=dict(
            title="Score",
            orientation="h",
            x=0.5,
            y=-0.1,
            xanchor="center",
            tickvals=[viz_df['Overall_Avg'].min(), viz_df['Overall_Avg'].max()],
            ticktext=["Low", "High"]
        )
    )
    
    return fig




In [142]:
def print_summary_statistics(viz_df):
    """Print summary statistics for normalized scores."""
    print("=" * 60)
    print("POPULATION-NORMALIZED RISK SCORE SUMMARY")
    print("=" * 60)
    print("\nFormula: Score_Normalized = Yearly_Avg / sqrt(Population_Thousands)")
    print("\n" + "-" * 60)
    
    for score_type in ['Drought', 'Flood', 'Overall']:
        norm_col = f'{score_type}_Norm'
        print(f"\n{score_type.upper()} (Normalized):")
        print(f"  Min: {viz_df[norm_col].min():.4f}")
        print(f"  Max: {viz_df[norm_col].max():.4f}")
        print(f"  Mean: {viz_df[norm_col].mean():.4f}")
        top = viz_df.nlargest(3, norm_col)[['Country', norm_col]]
        print(f"  Top 3: {', '.join(top['Country'].tolist())}")




In [143]:
# ============================================================
# MAIN EXECUTION
# ============================================================

print("Loading GeoJSON and data...")
geojson_data = load_geojson()
viz_df = load_and_normalize_data()



Loading GeoJSON and data...
Loaded GeoJSON with 258 countries
Loaded 55 countries with normalized scores


In [144]:
print("\nTop 10 countries by Overall Normalized Score:")
print(viz_df.nlargest(10, 'Overall_Norm')[
    ['Country', 'ISO3', 'Population_Thousands', 
     'Drought_Avg', 'Drought_Norm',
     'Flood_Avg', 'Flood_Norm',
     'Overall_Avg', 'Overall_Norm']
].to_string(index=False))

print_summary_statistics(viz_df)




Top 10 countries by Overall Normalized Score:
                Country ISO3  Population_Thousands  Drought_Avg  Drought_Norm  Flood_Avg  Flood_Norm  Overall_Avg  Overall_Norm
                Namibia  NAM              2467.077         5.78        0.1164       1.45      0.0292         3.62        0.0729
   Eswatini, Kingdom of  SWZ              1174.222         3.70        0.1080       1.05      0.0306         2.38        0.0695
               Djibouti  DJI              1082.344         4.10        0.1246       0.03      0.0009         2.06        0.0626
               Botswana  BWA              2523.541         4.20        0.0836       1.10      0.0219         2.65        0.0528
             Mauritania  MRT              4440.933         5.59        0.0839       1.35      0.0203         3.47        0.0521
                Lesotho  LSO              2239.604         4.33        0.0915       0.50      0.0106         2.42        0.0511
                Eritrea  ERI              3523.653       

In [145]:
print("\n" + "=" * 60)
print("GENERATING CHOROPLETH MAPS (saving as HTML)...")
print("=" * 60)




GENERATING CHOROPLETH MAPS (saving as HTML)...


In [146]:
# Create and save all maps
print("\n1. Drought Risk Map")
fig_drought = create_drought_map(viz_df, geojson_data)
fig_drought.write_html("drought_map.html", auto_open=True)
print("   Saved: drought_map.html")

print("\n2. Flood Risk Map")
fig_flood = create_flood_map(viz_df, geojson_data)
fig_flood.write_html("flood_map.html", auto_open=True)
print("   Saved: flood_map.html")

print("\n3. Overall Risk Map (Normalized)")
fig_overall = create_overall_map(viz_df, geojson_data)
fig_overall.write_html("overall_map.html", auto_open=True)
print("   Saved: overall_map.html")

print("\n4. Raw Overall Risk Map (for comparison)")
fig_raw = create_raw_overall_map(viz_df, geojson_data)
fig_raw.write_html("raw_overall_map.html", auto_open=True)
print("   Saved: raw_overall_map.html")

print("\n✓ All maps generated and saved as HTML files!")



1. Drought Risk Map


C:\Users\arun.gandhi\AppData\Local\Temp\ipykernel_20832\1580287007.py:7: DeprecationWarning:

*choropleth_mapbox* is deprecated! Use *choropleth_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



   Saved: drought_map.html

2. Flood Risk Map


C:\Users\arun.gandhi\AppData\Local\Temp\ipykernel_20832\1580287007.py:50: DeprecationWarning:

*choropleth_mapbox* is deprecated! Use *choropleth_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



   Saved: flood_map.html

3. Overall Risk Map (Normalized)


C:\Users\arun.gandhi\AppData\Local\Temp\ipykernel_20832\86295097.py:4: DeprecationWarning:

*choropleth_mapbox* is deprecated! Use *choropleth_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



   Saved: overall_map.html

4. Raw Overall Risk Map (for comparison)


C:\Users\arun.gandhi\AppData\Local\Temp\ipykernel_20832\86295097.py:49: DeprecationWarning:

*choropleth_mapbox* is deprecated! Use *choropleth_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



   Saved: raw_overall_map.html

✓ All maps generated and saved as HTML files!
